In [1]:
# Cleaning control group
import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob
os.getcwd()

samplerows=None

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
returned_df_1=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/Previous_Returned/CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz",compression="gzip",sep="|",dtype=str)
returned_df_2=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/Previous_Returned/CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz",compression="gzip",sep="|",dtype=str)

returned_df=returned_df_1.append(returned_df_2)
del returned_df_1
del returned_df_2

segment_previous_returned=returned_df['segment'].unique().tolist()

gc.collect()

7

In [4]:
# Also considered
my_Dec_new_signed_up_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/"
my_Dec_new_signed_up_list=[x for x in list(recursive_file_gen(my_Dec_new_signed_up_folder)) if ".csv" in x]
my_Dec_new_signed_up_list



['/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_P.csv',
 '/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_S.csv',
 '/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_2018-12-29/NewReward_2018-12-29_T.csv']

In [5]:
SP_all_after_0922_scoring="/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/New_Segments/"
SP_new_score_list=list(recursive_file_gen(SP_all_after_0922_scoring))
# set([x[len(x)-4:] for x in my_Dec_new_signed_up_list])

In [6]:
len(set([os.path.basename(x) for x in SP_new_score_list]))

92

In [7]:
all_file_to_be_loaded=SP_new_score_list+my_Dec_new_signed_up_list # 1229 appended
print(len(all_file_to_be_loaded))
print(len(set(all_file_to_be_loaded)))
print(len(set([os.path.basename(x) for x in all_file_to_be_loaded])))

95
95
95


In [8]:
all_file_to_be_loaded=SP_new_score_list+my_Dec_new_signed_up_list # 1229 appended
print(len(all_file_to_be_loaded))
print(len(set(all_file_to_be_loaded)))
print(len(set([os.path.basename(x) for x in all_file_to_be_loaded])))

# Load all segment without any drop_dups
df_all_new=pd.DataFrame()
all_file_to_be_loaded=sorted(all_file_to_be_loaded)
i_counter=0

for file_path in all_file_to_be_loaded:
    if os.path.basename(file_path)=="BL_lapsed19_48.csv":
        pass
    
    elif "Holiday_Shopper_" in os.path.basename(file_path): # To be appended below
        print(os.path.basename(file_path))
        pass
        
    else:
        df=pd.read_csv(file_path,dtype=str,usecols=['customer_id_hashed', 'email_address_hash', 'segment'],nrows=samplerows)
        # df=df[~df['segment'].isin(segment_previous_returned)]
        
        df_all_new=df_all_new.append(df)
    i_counter+=1
    if i_counter%10==2:
        print(i_counter,datetime.datetime.now())
    

95
95
95
2 2019-02-28 10:55:24.194774
12 2019-02-28 10:55:30.942834
Holiday_Shopper_Email_Non_Openers.csv
Holiday_Shopper_Email_Openers.csv
Holiday_Shopper_Full_File.csv
Holiday_Shopper_Purchase_Active.csv
Holiday_Shopper_Purchase_Inactive.csv
Holiday_Shopper_Toy_Purchase.csv
22 2019-02-28 10:55:33.635951
32 2019-02-28 10:55:39.114175
42 2019-02-28 10:55:41.829515
52 2019-02-28 10:56:06.589667
62 2019-02-28 10:56:53.513730
72 2019-02-28 10:57:14.452643
82 2019-02-28 10:57:42.243818
92 2019-02-28 10:58:21.630501


In [9]:
client_holiday='/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/New_Segments/upload_holidayshoppers/'
client_holiday_list=glob.glob(client_holiday+"*.csv")
client_holiday_df=pd.DataFrame()

for file_holiday in client_holiday_list:
    df=pd.read_csv(file_holiday,dtype=str,usecols=['customer_id_hashed', 'email_address_hash','segment'],nrows=samplerows)
    client_holiday_df=client_holiday_df.append(df)
    
# client_holiday_df_dedup=client_holiday_df.drop_duplicates('email_address_hash')
# client_holiday_df_dedup['segment']="Client_Holiday_Dedup"

In [10]:
df_all_new=df_all_new.append(client_holiday_df)
df_all_new.head(2)


,customer_id_hashed,email_address_hash,segment
0,73ffdaf7a7c7cc4be0351d7201c4444b745d1e0035a95f...,a909de3c3feee405ec29c9bd9e603d1220b88e64f625fc...,10.5 SOTF Cleveland
1,3adc60d511958bae1243d4f665ce763238fff72ac6a365...,abe5868177f707194a338dd98e4fd81306c087e3f8c5bd...,10.5 SOTF Cleveland


In [11]:
'''
lapsed_raw=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip"
                         ,nrows=None,sep="|",dtype=str,usecols=['customer_id_hashed','email_address_hash'])
lapsed_raw['segment']="Lapsed_20140826_20170226"
print(len(lapsed_raw.index))
lapsed_raw=lapsed_raw.drop_duplicates(['email_address_hash'])
print(len(lapsed_raw.index))

df_all_new=df_all_new.append(lapsed_raw)

print(df_all_new.shape)
print(len(df_all_new['email_address_hash'].unique()))
'''

'\nlapsed_raw=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip"\n                         ,nrows=None,sep="|",dtype=str,usecols=[\'customer_id_hashed\',\'email_address_hash\'])\nlapsed_raw[\'segment\']="Lapsed_20140826_20170226"\nprint(len(lapsed_raw.index))\nlapsed_raw=lapsed_raw.drop_duplicates([\'email_address_hash\'])\nprint(len(lapsed_raw.index))\n\ndf_all_new=df_all_new.append(lapsed_raw)\n\nprint(df_all_new.shape)\nprint(len(df_all_new[\'email_address_hash\'].unique()))\n'

In [12]:
# df_all_new=df_all_new[~df_all_new['segment'].isin(segment_previous_returned)]
df_all_new=df_all_new.drop_duplicates()
print(df_all_new.shape)
print(len(df_all_new['email_address_hash'].unique()))

(27288695, 3)
23024699


In [13]:
df_all_new['segment_first_2_chars']=df_all_new['segment'].apply(lambda x: x[:2])
control_groups=df_all_new[df_all_new['segment_first_2_chars']=="C_"]
other_groups=df_all_new[df_all_new['segment_first_2_chars']!="C_"]

control_groups.shape

(499880, 4)

In [14]:
del control_groups['segment_first_2_chars']
del df_all_new['segment_first_2_chars']
del other_groups['segment_first_2_chars']

In [15]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4'

In [16]:
df_all_new.to_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/all_SP_segments.csv',index=False)
control_groups.to_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/all_control_segments.csv',index=False)
other_groups.to_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/all_other_segments.csv',index=False)


In [17]:
df_1_control_groups=control_groups[~control_groups['customer_id_hashed'].isin(other_groups['customer_id_hashed'].tolist())]
print(df_1_control_groups.shape)

df_1_control_groups=df_1_control_groups[~df_1_control_groups['email_address_hash'].isin(other_groups['email_address_hash'].tolist())]
print(df_1_control_groups.shape)



(444432, 3)
(444430, 3)
